## Создаём словарь для CondBERT'а

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/bias_data.zip'

Archive:  /content/drive/MyDrive/bias_data.zip
   creating: bias_data/
   creating: bias_data/real_world_samples/
  inflating: bias_data/real_world_samples/news_left  
   creating: __MACOSX/
   creating: __MACOSX/bias_data/
   creating: __MACOSX/bias_data/real_world_samples/
  inflating: __MACOSX/bias_data/real_world_samples/._news_left  
  inflating: bias_data/real_world_samples/ibc_left  
  inflating: __MACOSX/bias_data/real_world_samples/._ibc_left  
  inflating: bias_data/real_world_samples/ibc_right  
  inflating: __MACOSX/bias_data/real_world_samples/._ibc_right  
  inflating: bias_data/real_world_samples/speeches  
  inflating: __MACOSX/bias_data/real_world_samples/._speeches  
  inflating: bias_data/real_world_samples/news_right  
  inflating: __MACOSX/bias_data/real_world_samples/._news_right  
  inflating: bias_data/real_world_samples/news_sensationalist_dramatic  
  inflating: __MACOSX/bias_data/real_world_samples/._news_sensationalist_dramatic  
  inflating: __MACOSX/bias_d

In [3]:
!git clone https://github.com/s-nlp/detox
! pip install -r detox/requirements.txt -q

Cloning into 'detox'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 180 (delta 50), reused 118 (delta 14), pack-reused 0
Receiving objects: 100% (180/180), 17.92 MiB | 15.27 MiB/s, done.
Resolving deltas: 100% (50/50), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 84.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.8/400.8 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 20.6 MB/s eta 0:00:00
  Prepari

In [4]:
import os
import sys

def add_sys_path(p):
    p = os.path.abspath(p)
    print(p)              
    if p not in sys.path:
        sys.path.append(p)

In [5]:
# adding the path to the condebert code root
add_sys_path('detox/emnlp2021/style_transfer/condBERT')

/content/detox/emnlp2021/style_transfer/condBERT


In [6]:
from importlib import reload

In [7]:
import condbert
reload(condbert)
from condbert import CondBertRewriter

In [8]:
!pip install flair keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.3/374.3 KB 7.5 MB/s eta 0:00:00
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached bpemb-0.3.4-py3-none-any.whl (19 kB)
  Using cached conllu-4.5.2-py2.py3-none-any.whl (16 kB)
  Using cached mpld3-0.3-py3-none-any.whl
  Using cached segtok-1.5.11-py3-none-any.whl (24 kB)
  Using cached Wikipedia_API-0.5.8-py3-none-any.whl (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 KB 13.0 MB/s eta 0:00:00
  Using cached Janome-0.4.2-py2.py3-none-any.whl (19.7 MB)
  Using cached sqlitedict-2.1.0-py3-none-any.whl
  Using cached langdetect-1.0.9-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 23.5 MB/s eta 0:00:00
  Using cached Deprecated-1.2.13-

In [ ]:
# !pip uninstall tokenizers transformers
# !pip install transformers

In [9]:
import choosers
reload(choosers)
from choosers import EmbeddingSimilarityChooser

In [10]:
import multiword
reload(multiword)
from multiword.masked_token_predictor_bert import MaskedTokenPredictorBert

# 0. Prerequisites

In [11]:
VOCAB_DIRNAME = 'bias_data/WNC'

# 1. Loading BERT

In [12]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import pickle
import os
from tqdm.auto import tqdm, trange

In [14]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device('cuda:0')
device = torch.device('cpu')

In [15]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [16]:
model = BertForMaskedLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
model.to(device);

# 2. Preparing the vocabularires.


- negative-words.txt
- positive-words.txt
- word2coef.pkl
- token_biasedness.txt

These files should be prepared once. 

In [19]:
biased_corpus_path = '/content/bias_data/WNC/biased.full'
norm_corpus_path = '/content/bias_data/WNC/neutral'

In [18]:
import pandas as pd

In [21]:
pd.set_option('display.max_colwidth', -1)

<ipython-input-21-0891b765a168>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [20]:
colnames = ['id','src_tok','tgt_tok','src_raw',
            'tgt_raw','src_POS_tags','tgt_parse_tags']
biased_df = pd.read_csv(biased_corpus_path, 
                        sep='\t', on_bad_lines='skip', names=colnames)
biased_df[['src_raw','tgt_raw']]

,src_raw,tgt_raw
0,"during the campaign, controversy erupted over ...","during the campaign, some pointed out alleged ..."
1,nicaea was convoked by the emperor constantine...,nicaea was convoked by the emperor constantine...
2,it was rather unfortunate that he vehemently o...,he vehemently opposed the budding indian scien...
3,dennis the menace is an american animated seri...,dennis the menace is an american animated seri...
4,"today, on large farms, motorcycles, dogs or me...","today, on large farms, motorcycles, dogs or pe..."
...,...,...
181468,"the most famous of these is dale earnhardt, wh...","arguably the most notable was dale earnhardt, ..."
181469,"because of the purity of the ancient breed, th...",because of the genetic prepotency of the ancie...
181470,"influenced by serge gainsbourg, the velvet und...","influenced by serge gainsbourg, the velvet und..."
181471,"northern indiana, for example, contains some o...","northern indiana, for example, contains the in..."


In [ ]:
with open('biased_df.txt', 'w') as f_out:
    f_out.write('\n'.join(biased_df['src_raw']))

In [ ]:
norm_df = pd.read_csv(norm_corpus_path, 
                      sep='\t', on_bad_lines='skip', names=colnames)
norm_df['src_raw'] = norm_df['src_raw'].dropna()
norm_df['src_raw'] = norm_df['src_raw'].apply(lambda x: str(x))
norm_df[['src_raw','tgt_raw']]

In [ ]:
with open('norm_df.txt', 'w') as f_out:
    f_out.write('\n'.join(norm_df['src_raw']))

In [ ]:
if not os.path.exists(VOCAB_DIRNAME):
    os.makedirs(VOCAB_DIRNAME)

In [ ]:
biased_corpus_path = 'biased_df.txt'
norm_corpus_path = 'norm_df.txt'

### 2.1 Preparing the DRG-like vocabularies

In [ ]:
import os
import argparse
import numpy as np
from tqdm import tqdm
from nltk import ngrams
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer



class NgramSalienceCalculator():
    def __init__(self, biased_corpus, norm_corpus, use_ngrams=False):
        ngrams = (1, 3) if use_ngrams else (1, 1)
        self.vectorizer = CountVectorizer(ngram_range=ngrams)

        biased_count_matrix = self.vectorizer.fit_transform(biased_corpus)
        self.biased_vocab = self.vectorizer.vocabulary_
        self.biased_counts = np.sum(biased_count_matrix, axis=0)

        norm_count_matrix = self.vectorizer.fit_transform(norm_corpus)
        self.norm_vocab = self.vectorizer.vocabulary_
        self.norm_counts = np.sum(norm_count_matrix, axis=0)

    def salience(self, feature, attribute='biased', lmbda=0.5):
        assert attribute in ['biased', 'norm']
        if feature not in self.biased_vocab:
            biased_count = 0.0
        else:
            biased_count = self.biased_counts[0, self.biased_vocab[feature]]

        if feature not in self.norm_vocab:
            norm_count = 0.0
        else:
            norm_count = self.norm_counts[0, self.norm_vocab[feature]]

        if attribute == 'biased':
            return (biased_count + lmbda) / (norm_count + lmbda)
        else:
            return (norm_count + lmbda) / (biased_count + lmbda)


In [ ]:
from collections import Counter
c = Counter()

for fn in tqdm([biased_corpus_path, norm_corpus_path]):
    with open(fn, 'r') as corpus:
        for line in corpus.readlines():
            for tok in line.strip().split():
                c[tok] += 1

print(len(c))

In [ ]:
vocab = {w for w, _ in c.most_common() if _ > 0}  # if we took words with > 1 occurences, vocabulary would be x2 smaller, but we'll survive this size
print(len(vocab))

In [ ]:
with open(biased_corpus_path, 'r') as biased_corpus, open(norm_corpus_path, 'r') as norm_corpus:
    corpus_biased = [' '.join([w if w in vocab else '<unk>' for w in line.strip().split()]) for line in biased_corpus.readlines()]
    corpus_norm = [' '.join([w if w in vocab else '<unk>' for w in line.strip().split()]) for line in norm_corpus.readlines()]

In [ ]:
neg_out_name = VOCAB_DIRNAME + '/biased-words.txt'
pos_out_name = VOCAB_DIRNAME + '/neut-words.txt'

In [ ]:
threshold = 4

In [ ]:
sc = NgramSalienceCalculator(corpus_biased, corpus_norm, False)
seen_grams = set()

with open(neg_out_name, 'w') as neg_out, open(pos_out_name, 'w') as pos_out:
    for gram in tqdm(set(sc.biased_vocab.keys()).union(set(sc.norm_vocab.keys()))):
        if gram not in seen_grams:
            seen_grams.add(gram)
            biased_salience = sc.salience(gram, attribute='biased')
            neut_salience = sc.salience(gram, attribute='norm')
            if biased_salience > threshold:
                neg_out.writelines(f'{gram}\n')
            elif neut_salience > threshold:
                pos_out.writelines(f'{gram}\n')

## 2.2 Evaluating word biasedness with a logistic regression

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))

In [ ]:
X_train = corpus_biased + corpus_norm
y_train = [1] * len(corpus_biased) + [0] * len(corpus_norm)
pipe.fit(X_train, y_train);

In [ ]:
coefs = pipe[1].coef_[0]
coefs.shape

In [ ]:
word2coef = {w: coefs[idx] for w, idx in pipe[0].vocabulary_.items()}

In [ ]:
import pickle
with open(VOCAB_DIRNAME + '/word2coef.pkl', 'wb') as f:
    pickle.dump(word2coef, f)

## 2.3 Labelling BERT tokens by biasedness

In [ ]:
from collections import defaultdict
bias_counter = defaultdict(lambda: 1)
nonbias_counter = defaultdict(lambda: 1)

for text in tqdm(corpus_biased):
    for token in tokenizer.encode(text[:512]):
        bias_counter[token] += 1
for text in tqdm(corpus_norm):
    for token in tokenizer.encode(text):
        nonbias_counter[token] += 1

In [ ]:
token_biasedness = [bias_counter[i] / (nonbias_counter[i] + bias_counter[i]) for i in range(len(tokenizer.vocab))]

In [ ]:
with open(VOCAB_DIRNAME + '/token_biasedness.txt', 'w') as f:
    for t in token_biasedness:
        f.write(str(t))
        f.write('\n')

# 3. Setting up the model

### 3.1 Loading the vocabularies

In [22]:
with open(VOCAB_DIRNAME + "/biased-words.txt", "r") as f:
    s = f.readlines()
negative_words = list(map(lambda x: x[:-1], s))

with open(VOCAB_DIRNAME + "/neut-words.txt", "r") as f:
    s = f.readlines()
positive_words = list(map(lambda x: x[:-1], s))

In [23]:
import pickle
with open(VOCAB_DIRNAME + '/word2coef.pkl', 'rb') as f:
    word2coef = pickle.load(f)

In [24]:
token_biasedness = []
with open(VOCAB_DIRNAME + '/token_biasedness.txt', 'r') as f:
    for line in f.readlines():
        token_biasedness.append(float(line))
token_biasedness = np.array(token_biasedness)
token_biasedness = np.maximum(0, np.log(1/(1/token_biasedness-1)))   # log odds ratio

# discourage meaningless tokens
for tok in ['.', ',', '-']:
    token_biasedness[tokenizer.encode(tok)][1] = 3

for tok in ['you']:
    token_biasedness[tokenizer.encode(tok)][1] = 0

In [36]:
def adjust_logits(logits, label=0):
    return logits - token_biasedness * 100 * (1 - 2 * label)

predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=250, 
                                     device=device, label=0, contrast_penalty=0.0, 
                                     logits_postprocessor=adjust_logits)

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_biasedness,
    predictor=predictor,
)

The model below is used for reranking BERT hypotheses and helps to increase semantic similarity by choosing the hypotheses with  embeddings similar to the orignal words. 

In [37]:
chooser = EmbeddingSimilarityChooser(sim_coef=10, tokenizer=tokenizer)

# 4. Finally, the inference

Parallel application of the model to all tokens, fast, but dirty. 

Application of the model to all the tokens sequentially, in the multiword mode. 

Parameters that could be tuned:
* The coeffincient in `adjust_logits` - the larger it is, the more the model avoids biased words
* The coefficient in `EmbeddingSimilarityChooser` - the larger it is, the more the model tries to preserve content 
* n_tokens - how many words can be generated from one
* n_top - how many BERT hypotheses are reranked

### Возьмём переведём 1000 примеров, чтобы оценить качество

In [27]:
test_path = '/content/bias_data/WNC/biased.word.test'

In [28]:
def translate(sent):
  return editor.replacement_loop(sent, verbose=False, 
                                 chooser=chooser, n_tokens=(1, 2, 3), n_top=10)

In [34]:
test_df = pd.read_csv(test_path, sep='\t', 
                      on_bad_lines='skip', names=colnames)

### Примеры модификации 

In [38]:
sentences = ['John McCain exposed as an unprincipled politician',
             'Marriage is a holy union of individuals.',
             'This includes amazing Americans like',
             'The authors expose on nutrition studies.',
             'Jewish forces overcome Arab militants.',
             ]

In [39]:
%%time
for sent in sentences:
  sent = sent.strip()
  first = editor.translate(sent, prnt=False)
  second = translate(sent)
  editor.replacement_loop(sent, verbose=False, 
                                   chooser=chooser, n_tokens=(1, 2, 3), n_top=10)
  print(first, second, '\n', sep ='\n')

john mccain exposed as an unprincipled .
john mccain exposed as an unprincipled democrat .


marriage is a holy union of individuals .
marriage is a holy union of individuals .


this includes all americans like
this includes african - americans like


the authors feature on nutrition studies .
the authors feature on nutrition studies .


. forces overcome arab militants .
israeli defense forces overcome arab rebel forces .


CPU times: user 1min 11s, sys: 134 ms, total: 1min 11s
Wall time: 1min 16s


In [ ]:
translations = []
for sent in tqdm(test_df['src_raw']):
  translations.append(translate(sent))

In [1]:
with open('translations.txt', 'r') as tr:
     text = tr.readlines()